# Converting a LangGraph Workflow into a Codon Workload

This notebook shows how to take an existing LangGraph graph built with LangChain components and wrap it with `CodonWorkload` using the `LangGraphWorkloadAdapter`. We get instrumentation, audit trails, logic IDs, and the token runtime without changing the LangGraph logic.

> **Colab friendly** – install LangChain, LangGraph, and Codon SDK (from source) before running the cells.

## 0. Setup

Install dependencies and configure environment variables. Adjust paths as needed.

In [ ]:
!pip install --quiet langchain langgraph langchain-openai openai tiktoken

import os
import sys

REPO_PATH = os.environ.get("CODON_SDK_PATH", "/content/codon-sdk")
if f"{REPO_PATH}/sdk/src" not in sys.path:
    sys.path.append(f"{REPO_PATH}/sdk/src")

# Replace with your own key or use Colab secrets
os.environ.setdefault("OPENAI_API_KEY", "sk-REPLACE_ME")
os.environ.setdefault("ORG_NAMESPACE", "codon-langgraph-demo")

## 1. Build an Existing LangGraph Workflow

We design a simple research workflow using LangChain tools:
- Planner synthesises a plan.
- Researcher expands insights.
- Writer produces a summary.
- Critic reviews and loop back if needed.

In [ ]:
from typing import Dict
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

from typing import TypedDict
from langgraph.graph import StateGraph

llm_planner = ChatOpenAI(model="gpt-4.1-mini")
llm_researcher = ChatOpenAI(model="gpt-4.1-mini")
llm_writer = ChatOpenAI(model="gpt-4.1-mini")
llm_critic = ChatOpenAI(model="gpt-4.1-mini")

async def planner(state: Dict):
    topic = state["topic"]
    prompt = f"Draft a research plan for {topic}. Keep it short."
    result = await llm_planner.ainvoke([HumanMessage(content=prompt)])
    return {"plan": result.content}

async def researcher(state: Dict):
    prompt = (
        "Given this plan, provide three bullet insights.
"
        f"Plan:
{state['plan']}"
    )
    result = await llm_researcher.ainvoke([HumanMessage(content=prompt)])
    return {"insights": result.content}

async def writer(state: Dict):
    prompt = (
        "Write a 120-word executive summary using the plan and insights.
"
        f"Plan: {state['plan']}
"
        f"Insights: {state['insights']}"
    )
    result = await llm_writer.ainvoke([HumanMessage(content=prompt)])
    return {"draft": result.content}

async def critic(state: Dict):
    prompt = (
        "Review this draft. Return JSON with fields 'decision' and 'feedback'.
"
        "Decision must be ACCEPT or REVISION."
        f"Draft: {state['draft']}"
    )
    result = await llm_critic.ainvoke([HumanMessage(content=prompt)])
    return {"critique": result.content}

async def finalize(state: Dict):
    return {"summary": state['draft'], "critique": state.get('critique', '')}

class ResearchState(TypedDict, total=False):
    topic: str
    plan: str
    insights: str
    draft: str
    critique: str

graph = StateGraph(ResearchState)
graph.add_node("planner", planner)
graph.add_node("researcher", researcher)
graph.add_node("writer", writer)
graph.add_node("critic", critic)
graph.add_node("finalize", finalize)

# Edges
graph.add_edge("planner", "researcher")
graph.add_edge("researcher", "writer")
graph.add_edge("writer", "critic")
# cycle: critic -> writer if revision required
graph.add_edge("critic", "writer")
# finalize when critic approves
graph.add_edge("critic", "finalize")

graph.set_entry_point("planner")
graph.set_finish_point("finalize")

## 2. Run the LangGraph Workflow Directly

Before wrapping with Codon, let's run the LangGraph graph to confirm it works.

In [ ]:
compiled_graph = graph.compile()
initial_state = {"topic": "Community gardens and urban wellbeing"}
result = await compiled_graph.ainvoke(initial_state)
result

## 3. Convert to Codon Workload

Using `LangGraphWorkloadAdapter` we can wrap the graph with zero manual instrumentation.

In [ ]:
from codon.instrumentation.langgraph import (
    initialize_telemetry,
    LangGraphWorkloadAdapter,
)

initialize_telemetry(service_name="codon-langgraph-notebook")

codon_workload = LangGraphWorkloadAdapter.from_langgraph(
    graph,
    name="LangGraphResearchAgent",
    version="0.1.0",
    description="Wrapped LangGraph research agent",
    tags=["langgraph", "demo"],
    role_overrides={
        "planner": "planner",
        "researcher": "analyst",
        "writer": "author",
        "critic": "qa",
        "finalize": "publisher",
    },
)
codon_workload

## 4. Execute the Codon Workload

Now we execute the workload with the same initial state. Tokens drive the LangGraph nodes under the hood, and we capture a ledger plus telemetry.

In [ ]:
report = await codon_workload.execute_async({"state": initial_state}, deployment_id="notebook-demo", max_steps=40)final_entry = report.node_results("finalize")[-1]final_entry

In [ ]:
print("Final summary:
", final_entry["summary"][:400], "
...")
print("Critique:
", final_entry["critique"])
print("Iterations:", report.context.get("iteration"))
print("Ledger entries:", len(report.ledger))

### Inspect Instruments

```
report.ledger[:5]
```
Use the ledger to review every node activation and token movement.

In [ ]:
for event in report.ledger[:10]:
    print(event.event_type, event.source_node, "->", event.target_node)

## 5. Benefits Recap
- **Telemetry**: `track_node` spans emitted automatically via the adapter.
- **Audit trail**: `ExecutionReport.ledger` stores a full tape of the run.
- **Logic IDs**: deterministic hashing of the LangGraph structure for idempotency.
- **Developer ergonomics**: one call to `from_langgraph` replaces manual instrumentation.

Next steps: explore the roadmap for persistence (`docs/vision/codon-workload-design-philosophy.md`) and the design guidelines for adapters (`docs/guides/workload-mixin-guidelines.md`).